## Analysis and Modeling of San Francisco Crime Data

### In this notebook, Spark SQL and dataframe was used for big data analysis and OLAP on SF crime data. (https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-Historical-2003/tmnf-yvry).

In [3]:
from csv import reader
from pyspark.sql import Row 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import warnings

import os
os.environ["PYSPARK_PYTHON"] = "python3"


In [4]:
# https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD
import urllib.request
urllib.request.urlretrieve("https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD", "/tmp/myxxxx.csv")
dbutils.fs.mv("file:/tmp/myxxxx.csv", "dbfs:/sf_crime_analysis/data/sf_03_18.csv")
display(dbutils.fs.ls("dbfs:/sf_crime_analysis/data/"))

path,name,size
dbfs:/sf_crime_analysis/data/sf_03_18.csv,sf_03_18.csv,559169754


In [5]:
data_path = "dbfs:/sf_crime_analysis/data/sf_03_18.csv"

### Solove  big data problems via Spark

#### Get dataframe and sql

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("crime analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df_opt1 = spark.read.format("csv").option("header", "true").load(data_path)
display(df_opt1)
df_opt1.createOrReplaceTempView("sf_crime")

PdId,IncidntNum,Incident Code,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,location,SF Find Neighborhoods 2 2,Current Police Districts 2 2,Current Supervisor Districts 2 2,Analysis Neighborhoods 2 2,DELETE - Fire Prevention Districts 2 2,DELETE - Police Districts 2 2,DELETE - Supervisor Districts 2 2,DELETE - Zip Codes 2 2,DELETE - Neighborhoods 2 2,DELETE - 2017 Fix It Zones 2 2,Civic Center Harm Reduction Project Boundary 2 2,Fix It Zones as of 2017-11-06 2 2,DELETE - HSOC Zones 2 2,Fix It Zones as of 2018-02-07 2 2,"CBD, BID and GBD Boundaries as of 2017 2 2","Areas of Vulnerability, 2016 2 2",Central Market/Tenderloin Boundary 2 2,Central Market/Tenderloin Boundary Polygon - Updated 2 2,HSOC Zones as of 2018-06-05 2 2,OWED Public Spaces 2 2,Neighborhoods 2
11087941463010,110879414,63010,WARRANTS,WARRANT ARREST,Friday,07/01/2011,08:00,NORTHERN,"ARREST, BOOKED",3400 Block of PIERCE ST,-122.439757857075,37.8021507619169,POINT (-122.43975785707501 37.8021507619169),17,4,6,13,13,9,1,57,17,null,null,null,null,null,null,1,null,null,null,null,17
5117770316010,051177703,16010,DRUG/NARCOTIC,POSSESSION OF MARIJUANA,Tuesday,10/18/2005,14:30,TENDERLOIN,"ARREST, BOOKED",0 Block of UNITEDNATIONS PZ,-122.414317857881,37.7799444052046,POINT (-122.414317857881 37.7799444052046),21,5,10,36,7,10,9,28852,36,3,1,3,1,3,6,2,1,1,1,39,21
5011209707055,050112097,07055,VEHICLE THEFT,RECOVERED VEHICLE - STOLEN OUTSIDE SF,Saturday,01/29/2005,13:45,BAYVIEW,NONE,1500 Block of KIRKWOOD AV,-122.388798895151,37.7375755833256,POINT (-122.38879889515101 37.7375755833256),86,2,9,1,10,3,8,58,1,null,null,null,null,null,null,2,null,null,null,null,86
11044468164020,110444681,64020,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Thursday,06/02/2011,02:52,CENTRAL,PSYCHOPATHIC CASE,900 Block of COLUMBUS AV,-122.414354301151,37.8031089840376,POINT (-122.41435430115101 37.8031089840376),107,6,3,32,3,1,10,308,32,null,null,null,null,null,null,1,null,null,null,null,107
3038311107024,030383111,07024,VEHICLE THEFT,STOLEN TRAILER,Saturday,02/01/2003,08:00,BAYVIEW,NONE,1500 Block of BAY SHORE BL,-122.401096851568,37.7245556697717,POINT (-122.401096851568 37.7245556697717),86,2,9,1,10,3,8,58,1,null,null,null,null,null,null,2,null,null,null,null,86
6018621406243,060186214,06243,LARCENY/THEFT,PETTY THEFT FROM LOCKED AUTO,Sunday,02/12/2006,17:00,CENTRAL,NONE,400 Block of DAVIS CT,-122.398187664281,37.7967148853927,POINT (-122.39818766428101 37.7967148853927),77,6,3,8,3,1,10,28860,6,null,null,null,null,null,null,1,null,null,null,null,77
9606640606374,096066406,06374,LARCENY/THEFT,GRAND THEFT OF PROPERTY,Thursday,08/06/2009,22:00,RICHMOND,NONE,2000 Block of BAKER ST,-122.444143357971,37.7899090887789,POINT (-122.44414335797102 37.7899090887789),102,8,6,30,13,6,1,29490,27,null,null,null,null,null,null,1,null,null,null,null,102
10037601606304,100376016,06304,LARCENY/THEFT,GRAND THEFT FROM A BUILDING,Friday,04/23/2010,16:30,MISSION,NONE,2600 Block of MISSION ST,-122.41860030589,37.7546255872838,POINT (-122.41860030589001 37.7546255872838),53,3,2,20,2,4,7,28859,19,null,null,null,3,null,null,2,null,null,3,null,53
5118607706244,051186077,06244,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Thursday,10/20/2005,11:00,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403404791479,37.775420706711,POINT (-122.40340479147902 37.775420706711),32,1,10,34,14,2,9,28853,34,null,null,null,null,null,null,2,null,null,null,null,32
12038512205053,120385122,05053,BURGLARY,"BURGLARY OF STORE, UNLAWFUL ENTRY",Thursday,05/03/2012,10:00,SOUTHERN,NONE,600 Block of 2ND ST,-122.39123835272,37.7812532736369,POINT (-122.39123835272001 37.7812532736369),31,1,10,8,6,2,9,28856,6,null,null,null,null,null,null,1,null,null,null,null,31


#### OLAP: number counts for different crime categories

In [10]:
df_category = df_opt1.groupBy('category').count().orderBy('count', ascending=False)
display(df_category)

category,count
LARCENY/THEFT,477975
OTHER OFFENSES,303027
NON-CRIMINAL,236937
ASSAULT,191384
VEHICLE THEFT,126228
DRUG/NARCOTIC,117875
VANDALISM,114718
WARRANTS,99821
BURGLARY,91067
SUSPICIOUS OCC,79740


#### Solving the same problem by Spark SQL

In [12]:
#Spark SQL based
crimeCategory = spark.sql("SELECT  category, COUNT(*) AS Count FROM sf_crime GROUP BY category ORDER BY Count DESC")
display(crimeCategory)

category,Count
LARCENY/THEFT,477975
OTHER OFFENSES,303027
NON-CRIMINAL,236937
ASSAULT,191384
VEHICLE THEFT,126228
DRUG/NARCOTIC,117875
VANDALISM,114718
WARRANTS,99821
BURGLARY,91067
SUSPICIOUS OCC,79740


In [13]:
crimes_pd_df = crimeCategory.toPandas()

#### OLAP: Crime counts for different district

In [15]:
#Spark SQL based
crimeDistrict = spark.sql("SELECT PdDistrict as District , COUNT(*) AS Count FROM sf_crime GROUP BY District ORDER BY Count DESC")
display(crimeDistrict)

District,Count
SOUTHERN,394234
MISSION,293072
NORTHERN,269229
CENTRAL,223962
BAYVIEW,210729
TENDERLOIN,189034
INGLESIDE,186645
TARAVAL,158710
PARK,121070
RICHMOND,114267


#### OLAP: Crime counts for each Sunday at downtown San Francisco
SF downtown is defiend by the range of spatial location: X (-122.4213, -122.4313), Y (37.7540, 37.7740)

In [17]:
crimeSundayDowntown = spark.sql("select Date, substring(Date,7) as Year, substring(Date,1,2) as Month, DayOfWeek, COUNT(*) AS Count FROM sf_crime WHERE DayOfWeek = 'Sunday' \
                          AND X > -122.4313 AND X < -122.4213 AND Y > 37.7540 AND Y < 37.7740 \
                          GROUP BY Date, Year, Month, DayOfWeek ORDER BY Year, Month")
display(crimeSundayDowntown)

Date,Year,Month,DayOfWeek,Count
01/12/2003,2003,01,Sunday,20
01/26/2003,2003,01,Sunday,13
01/05/2003,2003,01,Sunday,14
01/19/2003,2003,01,Sunday,17
02/16/2003,2003,02,Sunday,13
02/23/2003,2003,02,Sunday,14
02/02/2003,2003,02,Sunday,14
02/09/2003,2003,02,Sunday,22
03/02/2003,2003,03,Sunday,16
03/09/2003,2003,03,Sunday,8


#### OLAP: Analysis of crime counts in each month from 2015 to 2018

In [19]:
from pyspark.sql.functions import hour, date_format, to_date, month, year
# add new columns to convert Date to date format
df_new = df_opt1.withColumn("IncidentDate",to_date(df_opt1.Date, "MM/dd/yyyy")) 
# extract month and year from incident date
df_new = df_new.withColumn('Month',month(df_new['IncidentDate']))
df_new = df_new.withColumn('Year', year(df_new['IncidentDate']))
display(df_new)

PdId,IncidntNum,Incident Code,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,location,SF Find Neighborhoods 2 2,Current Police Districts 2 2,Current Supervisor Districts 2 2,Analysis Neighborhoods 2 2,DELETE - Fire Prevention Districts 2 2,DELETE - Police Districts 2 2,DELETE - Supervisor Districts 2 2,DELETE - Zip Codes 2 2,DELETE - Neighborhoods 2 2,DELETE - 2017 Fix It Zones 2 2,Civic Center Harm Reduction Project Boundary 2 2,Fix It Zones as of 2017-11-06 2 2,DELETE - HSOC Zones 2 2,Fix It Zones as of 2018-02-07 2 2,"CBD, BID and GBD Boundaries as of 2017 2 2","Areas of Vulnerability, 2016 2 2",Central Market/Tenderloin Boundary 2 2,Central Market/Tenderloin Boundary Polygon - Updated 2 2,HSOC Zones as of 2018-06-05 2 2,OWED Public Spaces 2 2,Neighborhoods 2,IncidentDate,Month,Year
11087941463010,110879414,63010,WARRANTS,WARRANT ARREST,Friday,07/01/2011,08:00,NORTHERN,"ARREST, BOOKED",3400 Block of PIERCE ST,-122.439757857075,37.8021507619169,POINT (-122.43975785707501 37.8021507619169),17,4,6,13,13,9,1,57,17,null,null,null,null,null,null,1,null,null,null,null,17,2011-07-01,7,2011
5117770316010,051177703,16010,DRUG/NARCOTIC,POSSESSION OF MARIJUANA,Tuesday,10/18/2005,14:30,TENDERLOIN,"ARREST, BOOKED",0 Block of UNITEDNATIONS PZ,-122.414317857881,37.7799444052046,POINT (-122.414317857881 37.7799444052046),21,5,10,36,7,10,9,28852,36,3,1,3,1,3,6,2,1,1,1,39,21,2005-10-18,10,2005
5011209707055,050112097,07055,VEHICLE THEFT,RECOVERED VEHICLE - STOLEN OUTSIDE SF,Saturday,01/29/2005,13:45,BAYVIEW,NONE,1500 Block of KIRKWOOD AV,-122.388798895151,37.7375755833256,POINT (-122.38879889515101 37.7375755833256),86,2,9,1,10,3,8,58,1,null,null,null,null,null,null,2,null,null,null,null,86,2005-01-29,1,2005
11044468164020,110444681,64020,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Thursday,06/02/2011,02:52,CENTRAL,PSYCHOPATHIC CASE,900 Block of COLUMBUS AV,-122.414354301151,37.8031089840376,POINT (-122.41435430115101 37.8031089840376),107,6,3,32,3,1,10,308,32,null,null,null,null,null,null,1,null,null,null,null,107,2011-06-02,6,2011
3038311107024,030383111,07024,VEHICLE THEFT,STOLEN TRAILER,Saturday,02/01/2003,08:00,BAYVIEW,NONE,1500 Block of BAY SHORE BL,-122.401096851568,37.7245556697717,POINT (-122.401096851568 37.7245556697717),86,2,9,1,10,3,8,58,1,null,null,null,null,null,null,2,null,null,null,null,86,2003-02-01,2,2003
6018621406243,060186214,06243,LARCENY/THEFT,PETTY THEFT FROM LOCKED AUTO,Sunday,02/12/2006,17:00,CENTRAL,NONE,400 Block of DAVIS CT,-122.398187664281,37.7967148853927,POINT (-122.39818766428101 37.7967148853927),77,6,3,8,3,1,10,28860,6,null,null,null,null,null,null,1,null,null,null,null,77,2006-02-12,2,2006
9606640606374,096066406,06374,LARCENY/THEFT,GRAND THEFT OF PROPERTY,Thursday,08/06/2009,22:00,RICHMOND,NONE,2000 Block of BAKER ST,-122.444143357971,37.7899090887789,POINT (-122.44414335797102 37.7899090887789),102,8,6,30,13,6,1,29490,27,null,null,null,null,null,null,1,null,null,null,null,102,2009-08-06,8,2009
10037601606304,100376016,06304,LARCENY/THEFT,GRAND THEFT FROM A BUILDING,Friday,04/23/2010,16:30,MISSION,NONE,2600 Block of MISSION ST,-122.41860030589,37.7546255872838,POINT (-122.41860030589001 37.7546255872838),53,3,2,20,2,4,7,28859,19,null,null,null,3,null,null,2,null,null,3,null,53,2010-04-23,4,2010
5118607706244,051186077,06244,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Thursday,10/20/2005,11:00,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403404791479,37.775420706711,POINT (-122.40340479147902 37.775420706711),32,1,10,34,14,2,9,28853,34,null,null,null,null,null,null,2,null,null,null,null,32,2005-10-20,10,2005
12038512205053,120385122,05053,BURGLARY,"BURGLARY OF STORE, UNLAWFUL ENTRY",Thursday,05/03/2012,10:00,SOUTHERN,NONE,600 Block of 2ND ST,-122.39123835272,37.7812532736369,POINT (-122.39123835272001 37.7812532736369),31,1,10,8,6,2,9,28856,6,null,null,null,null,null,null,1,null,null,null,null,31,2012-05-03,5,2012


In [20]:
years = [i for i in range(2013, 2019)]
df_years = df_new[df_new.Year.isin(years)]
crime_month_year = df_years.groupby(['Year', 'Month']).count().orderBy('Year','Month')
display(crime_month_year)

Year,Month,count
2013,1,12461
2013,2,11436
2013,3,12617
2013,4,12145
2013,5,12556
2013,6,12153
2013,7,12810
2013,8,13383
2013,9,13482
2013,10,13586


In [21]:
crime_category_year = df_years.groupby(['Category', 'Year']).count().orderBy(['Year', 'count'])
display(crime_category_year)

Category,Year,count
TREA,2013,4
PORNOGRAPHY/OBSCENE MAT,2013,5
"SEX OFFENSES, NON FORCIBLE",2013,16
GAMBLING,2013,22
BAD CHECKS,2013,26
EXTORTION,2013,30
LOITERING,2013,54
FAMILY OFFENSES,2013,59
BRIBERY,2013,69
SUICIDE,2013,73


#### OLAP: 
Analysis the number of crime with respect to different hours in certian days.
Sample dates chosen are 12/15/2013,12/15/2014, 12/15/2015, 12/15/2016, 12/15/2017.

In [23]:
import pyspark.sql.functions as F
df_hour = df_new.withColumn("Hour", F.hour(df_new.Time))
display(df_hour)
dates = ['12/15/2013','12/15/2014','12/15/2015','12/15/2016','12/15/2017']
df_days = df_hour[df_new.Date.isin(dates)]
crime_hourly = df_days.groupby('Hour','Date').count().orderBy('Hour','Date')
display(crime_hourly)

Hour,Date,count
0,12/15/2013,35
0,12/15/2014,17
0,12/15/2015,15
0,12/15/2016,22
0,12/15/2017,30
1,12/15/2013,14
1,12/15/2014,2
1,12/15/2015,6
1,12/15/2016,10
1,12/15/2017,12


#### OLAP: Analysis of the crime events in the three most dangerous districts

In [25]:
top3danger = spark.sql( "Select PdDistrict, count(*) as Count from sf_crime group by 1 order by 2 desc limit 3")
display(top3danger)

PdDistrict,Count
SOUTHERN,394234
MISSION,293072
NORTHERN,269229


In [26]:
danger_zones = ['SOUTHERN', 'MISSION', 'NORTHERN']
crime_danger_zones = df_hour.filter(df_hour.PdDistrict.isin(danger_zones)).groupby('Category','Hour').count().orderBy('Category','Hour')
display(crime_danger_zones)

Category,Hour,count
ARSON,0,84
ARSON,1,77
ARSON,2,84
ARSON,3,95
ARSON,4,74
ARSON,5,60
ARSON,6,50
ARSON,7,28
ARSON,8,34
ARSON,9,28


####Trend and Advise
By counting total crime numbers, the three most dangerous districts are SOUTHERN, MISSION, and NORTHERN.
Crime frequency is especially high during 5-7 pm, and very few cases are found around 5 am. For the local police departments, the suggestion is pay more attention during high risk hours, especially to theft and assult.

#### OLAP: Analysis of the resolution for different crime categories

In [29]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window
resolution_func = udf (lambda x: x != 'NONE')
percent_resolution_df = df_hour.withColumn('IsResolution', resolution_func(F.col('Resolution')))

percent_resolution_df = percent_resolution_df.groupBy('category', 'Resolution', 'IsResolution').count().withColumnRenamed('count', 'resolved').orderBy('category')
percent_resolution_df = percent_resolution_df.withColumn('total', F.sum('resolved').over(Window.partitionBy('category'))).withColumn('Percentage', F.col('resolved') / F.col('total') * 100).filter(percent_resolution_df.IsResolution == True).orderBy('Percentage', ascending=False)
             
display(percent_resolution_df)

category,Resolution,IsResolution,resolved,total,Percentage
WARRANTS,"ARREST, BOOKED",true,93092,99821,93.2589334909488
DRIVING UNDER THE INFLUENCE,"ARREST, BOOKED",true,4941,5652,87.42038216560509
STOLEN PROPERTY,"ARREST, BOOKED",true,9693,11450,84.65502183406115
DRUG/NARCOTIC,"ARREST, BOOKED",true,97961,117875,83.10583244962885
DRUNKENNESS,"ARREST, BOOKED",true,7471,9760,76.54713114754098
WEAPON LAWS,"ARREST, BOOKED",true,13334,21004,63.48314606741573
LOITERING,"ARREST, CITED",true,1432,2402,59.61698584512906
PROSTITUTION,"ARREST, CITED",true,9667,16501,58.584328222531965
BRIBERY,"ARREST, BOOKED",true,418,796,52.51256281407035
LIQUOR LAWS,"ARREST, BOOKED",true,2056,3941,52.16950012687135


In [30]:
total_resolution_df = df_hour.withColumn('IsResolution', resolution_func(F.col('Resolution')))
total_resolution_df = total_resolution_df.groupBy('category', 'Resolution', 'IsResolution').count().withColumnRenamed('count', 'resolved').orderBy('category')
total_resolution_df = total_resolution_df.withColumn('total', F.sum('resolved').over(Window.partitionBy('category'))).withColumn('Total', F.col('resolved')).filter(total_resolution_df.IsResolution == True).orderBy('Total', ascending=False)
display(total_resolution_df)

category,Resolution,IsResolution,resolved,Total
OTHER OFFENSES,"ARREST, BOOKED",true,115726,115726
DRUG/NARCOTIC,"ARREST, BOOKED",true,97961,97961
OTHER OFFENSES,"ARREST, CITED",true,95152,95152
WARRANTS,"ARREST, BOOKED",true,93092,93092
ASSAULT,"ARREST, BOOKED",true,62809,62809
NON-CRIMINAL,PSYCHOPATHIC CASE,true,27027,27027
LARCENY/THEFT,"ARREST, BOOKED",true,25136,25136
MISSING PERSON,LOCATED,true,19615,19615
WEAPON LAWS,"ARREST, BOOKED",true,13334,13334
BURGLARY,"ARREST, BOOKED",true,12765,12765
